In [1]:
import glob
import numpy as np
import pandas as pd
import vaex

from astropy.coordinates import SkyCoord
from astropy.time import Time
from m33_utils import open_and_select

In [2]:
df = pd.read_csv('../header_info.csv', index_col=0)
df['FILT'] = df['FILT_DET'].str.split('-').str[1]
coo = SkyCoord(df[['RAC','DECC']].values, frame='icrs', unit='deg')
df['COO_STR'] = coo.to_string('hmsdms', format='latex', precision=0)
df['RA_STR'] = df['COO_STR'].str.split().str[0]
df['DEC_STR'] = df['COO_STR'].str.split().str[1]
t = Time(df.EXPSTART, format='mjd')
t.format = 'iso'
t.precision = 0
df['EXPDATE'] = t.value
df['EXPT'] = df['EXPTIME'].round(2)
df['ORIENT'] = df['ORIENTAT'].round(4)
cols = ['TARGNAME','RA_STR','DEC_STR','EXPDATE','EXPT','INSTRUME','APERTURE','FILT','ORIENT']
table_obs = df[cols].sort_values(by=['TARGNAME','EXPDATE'])

table_obs.to_csv('tables_full/table_obs.csv', index=False)
with open('tables/table_obs.tex', 'w') as f:
    f.write(table_obs[table_obs.TARGNAME.str.startswith('M33-B01-F01')
                     ].to_latex(index=False, escape=False))

In [ ]:
ds = vaex.open('../legacy_phot/M33_full_matched.hdf5')

filters = ['F275W','F336W','F475W','F814W','F110W','F160W']
for f in filters:
    gst_tf = ds[f'{f}_GST_FLAG'].astype(str).str.replace('0','F').str.replace('1','T')
    ds.add_virtual_column(f'{f}_GST', gst_tf)
cols = ['RA', 'DEC']
cols += [f'{f}_{q}' for f in filters for q in ['VEGA','SNR','GST']]
ds_phot = ds[cols].fillna(99.999)
for c in ds_phot.get_column_names(regex='.*?((VEGA)|(SNR))'):
    ds_phot[c] = ds_phot[c].apply(lambda x: np.round(x, 3))
ds_phot.export_csv('tables_full/table_phot.csv')
df_phot = ds_phot[5:15].to_pandas_df()
df_phot.rename(lambda x: x.split('_')[0] if x.endswith('VEGA') else x.split('_')[-1].replace('SNR','S/N'),
               axis='columns', inplace=True)

with open('tables/table_phot.tex', 'w') as f:
    f.write(df_phot.to_latex(index=False))
    
ds.close_files()

In [ ]:
# img1:img30-wfc3/ir, img31:img70-wfc3/uvis, img71:-acs/wfc, 
df_par = pd.read_csv('../M33_B03_SW/M33_B03_SW_3.param', sep='=', comment='#',
                     index_col=0, names=['Parameter','Value'])
df_par.index = df_par.index.str.strip()
df_par['Value'] = df_par['Value'].astype(str).str.strip()
df_par.rename(lambda x: x.replace('img1_', 'IR_'), axis='rows', inplace=True)
df_par.rename(lambda x: x.replace('img31_', 'UVIS_'), axis='rows', inplace=True)
df_par.rename(lambda x: x.replace('img71_', 'WFC_'), axis='rows', inplace=True)
df_par.drop(df_par.filter(regex='img.*?_', axis='rows').index, inplace=True)
df_par.drop(df_par.filter(regex='_file', axis='rows').index, inplace=True)
df_par.drop(['Nimg', 'xytfile', 'xytpsf', 'psfstars', 'UsePhot', 'photsec'], inplace=True)
df_par = df_par.assign(Detector='All')
df_par.loc[df_par.filter(regex='(^IR_)|(^WFC3IR)', axis='rows').index, 'Detector'] = 'IR'
df_par.loc[df_par.filter(regex='(^UVIS_)|(^WFC3UVIS)', axis='rows').index, 'Detector'] = 'UVIS'
df_par.loc[df_par.filter(regex='(^WFC_)|(^ACS)', axis='rows').index, 'Detector'] = 'WFC'
df_par.loc['WFC3useCTE', 'Detector'] = 'UVIS/IR'
df_par.reset_index(inplace=True)
df_par.loc[15, ['Detector', 'Parameter']] = ['All', 'apsky']
df_par.drop([16,17], inplace=True)
df_par['Parameter'] = df_par['Parameter'].str.replace('IR_','').str.replace('UVIS_','').str.replace('WFC_','')

with open('tables/table_par.tex', 'w') as f:
    f.write(df_par[['Detector','Parameter','Value']].to_latex(index=False))

In [ ]:
def read_artstars(infile):
    ds = open_and_select(infile)
    ds['DPOS'] = ds['((X_OUT-X_IN)**2 + (Y_OUT-Y_IN)**2)**0.5']
    for f in ds.get_column_names(regex='F.*?W_IN'):
        filt = f.split('_')[0]
        ds.add_column(f'{filt}_DELTA', ds[f'{filt}_VEGA-{filt}_IN'].evaluate())
        selection = f'({filt}_GST) & (DPOS < 2) & (abs({filt}_DELTA) < 2)'
        ds.select(selection, name=f'{filt}_detected')
        is_detected = ds.evaluate_selection_mask(f'{filt}_detected').astype(int)
        ds.add_column(f'{filt}_DET', is_detected)
    return ds

ds = read_artstars(glob.glob('../artstars/M33_B0?_??_fake_final.hdf5'))

filters = ['F275W','F336W','F475W','F814W','F110W','F160W']
for f in filters:
    gst_tf = ds[f'{f}_DET'].astype(str).str.replace('0','F').str.replace('1','T')
    ds.add_virtual_column(f'{f}_DT', gst_tf)
cols = ['RA', 'DEC']
cols += [f'{f}_{q}' for f in filters for q in ['IN', 'DELTA', 'SNR', 'DT']]
ds_ast = ds[cols].fillna(99.999)
for c in ds_ast.get_column_names(regex='.*?((VEGA)|(SNR)|(DELTA)|(IN))'):
    ds_ast[c] = ds_ast[c].apply(lambda x: np.round(x, 3))
ds_ast.export_csv('tables_full/table_asts.csv')
df_ast = ds_ast[5:15].to_pandas_df()

def rename(x):
    if x.endswith('IN'):
        name = x.split('_')[0] + ' in'
    elif x.endswith('DELTA'):
        name = 'Out - in'
    elif x.endswith('DT'):
        name = 'Recovered'
    else:
        name = x.split('_')[-1].replace('SNR','S/N')
    return name

df_ast.rename(rename, axis='columns', inplace=True)

with open('tables/table_asts.tex', 'w') as f:
    f.write(df_ast.to_latex(index=False))
    
ds.close_files()